<a href="https://colab.research.google.com/github/jullazarovych/objectdetectionML/blob/main/yolov5_mask_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Git**

In [ ]:
!mkdir -p ~/.ssh
!chmod 700 ~/.ssh

config_content = """
Host github.com
    HostName github.com
    User git
    IdentityFile ~/.ssh/id_rsa
    StrictHostKeyChecking no
"""

with open('/root/.ssh/config', 'w') as f:
    f.write(config_content)

!chmod 600 ~/.ssh/config
!chmod 600 ~/.ssh/id_rsa

In [ ]:
!ssh -T git@github.com

In [ ]:
!git clone git@github.com:jullazarovych/objectdetectionML.git

!git config --global user.name "yuliiaLazarovych"
!git config --global user.email "yuliia.lazarovych.22@pnu.edu.ua"

In [ ]:
!git pull origin main

In [ ]:
%cd /content/objectdetectionML

In [ ]:
status = !git status --porcelain
merge_head = !ls .git/MERGE_HEAD 2>/dev/null || echo "no merge"

if not status and "no merge" in merge_head[0]:
    print("no conflicts")
elif "MERGE_HEAD" in str(merge_head):
    print("uncomplited merge")
    !git status
else:
    print("there are changes, but not conflicts")
    !git status

In [ ]:
!cp "/content/drive/MyDrive/Colab Notebooks/yolov5_mask_detection.ipynb" /content/objectdetectionML/
%cd /content/objectdetectionML
!git add -f yolov5_mask_detection.ipynb
!git commit -m "added training&testing on ordinary dataset & with augmentation, LIME explanation included"
!git push origin main

In [ ]:
#!cp -r runs /content/drive/MyDrive/yolo_data/

# **Installing & importing YOLOv5 & necessary utils**

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
import os
from google.colab import drive
from pathlib import Path
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import numpy as np
import seaborn as sns
import xml.etree.ElementTree as ET
import shutil
from PIL import Image
import random
from sklearn.model_selection import train_test_split
from skimage.segmentation import mark_boundaries, slic
%matplotlib inline

In [ ]:
drive.mount('/content/drive', force_remount=True)

In [ ]:
#!unzip /content/drive/MyDrive/yolo_data/archive.zip -d /content/drive/MyDrive/yolo_data/

# **Testing YOLOv5 on standart COCO categories**

In [ ]:
model = YOLO('yolov5m.pt')

In [ ]:
image_test_folder = Path('/content/drive/MyDrive/yolo_data/test_data')
image_files = list(image_test_folder.glob("*.jpg")) + list(image_test_folder.glob("*.jpeg"))

In [ ]:
random.shuffle(image_files)

for image_file in image_files[:10]:
    results = model(image_file)
    img_with_boxes = results[0].plot()

    plt.figure(figsize=(10, 8))
    plt.imshow(cv2.cvtColor(img_with_boxes, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# **Preparing annotations in YOLO format, splitting dataset**

In [ ]:
xml_folder = '/content/drive/MyDrive/yolo_data/annotations'
txt_folder = '/content/drive/MyDrive/yolo_data/labelfull'
image_folder = '/content/drive/MyDrive/yolo_data/images'
os.makedirs(txt_folder, exist_ok=True)

In [ ]:
classes = ["with_mask", "without_mask", "mask_weared_incorrect"]
output_dir = "/content/drive/MyDrive/yolo_data"

In [ ]:
def convert_bbox(size, box):
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    xmin, ymin, xmax, ymax = box
    x_center = (xmin + xmax) / 2.0
    y_center = (ymin + ymax) / 2.0
    w = xmax - xmin
    h = ymax - ymin
    return (x_center * dw, y_center * dh, w * dw, h * dh)

In [ ]:
def convert_annotations_to_yolo(xml_folder, txt_folder):
    os.makedirs(txt_folder, exist_ok=True)

    for file in os.listdir(xml_folder):
        if not file.endswith(".xml"):
            continue

        in_file = os.path.join(xml_folder, file)
        tree = ET.parse(in_file)
        root = tree.getroot()

        size = root.find('size')
        w = int(size.find('width').text)
        h = int(size.find('height').text)
        filename = root.find('filename').text
        txt_filename = os.path.splitext(filename)[0] + ".txt"
        txt_path = os.path.join(txt_folder, txt_filename)

        with open(txt_path, "w") as out_file:
            for obj in root.findall('object'):
                cls_name = obj.find('name').text
                if cls_name not in classes:
                    continue
                cls_id = classes.index(cls_name)
                xmlbox = obj.find('bndbox')
                b = (
                    int(xmlbox.find('xmin').text),
                    int(xmlbox.find('ymin').text),
                    int(xmlbox.find('xmax').text),
                    int(xmlbox.find('ymax').text)
                )
                bb = convert_bbox((w, h), b)
                out_file.write(f"{cls_id} {bb[0]:.6f} {bb[1]:.6f} {bb[2]:.6f} {bb[3]:.6f}\n")

In [ ]:
convert_annotations_to_yolo(xml_folder, txt_folder)

In [ ]:
images = [f for f in os.listdir(image_folder) if f.endswith('.jpg') or f.endswith('.png')]
train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

In [ ]:
def move_files(file_list, src_img, src_lbl, dst_img, dst_lbl):
    os.makedirs(dst_img, exist_ok=True)
    os.makedirs(dst_lbl, exist_ok=True)
    for file in file_list:
        shutil.move(os.path.join(src_img, file), os.path.join(dst_img, file))
        label_file = os.path.splitext(file)[0] + '.txt'
        shutil.move(os.path.join(src_lbl, label_file), os.path.join(dst_lbl, label_file))

In [ ]:
txt_folder_out='/content/drive/MyDrive/yolo_data/labels'

In [ ]:
os.makedirs(txt_folder_out, exist_ok=True)

In [ ]:
"""move_files(train_imgs,
           image_folder, txt_folder,
           os.path.join(output_dir, "images/train"),
           os.path.join(output_dir, "labels/train"))

move_files(val_imgs,
           image_folder, txt_folder,
           os.path.join(output_dir, "images/val"),
           os.path.join(output_dir, "labels/val"))""

In [ ]:
def copy_labels_for_images(img_dir, label_src, label_dst):
    os.makedirs(label_dst, exist_ok=True)

    image_files = [f for f in os.listdir(img_dir)
                   if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]

    copied_count = 0

    for img_name in image_files:
        base_name = os.path.splitext(img_name)[0]

        label_filename = base_name + ".txt"
        label_src_path = os.path.join(label_src, label_filename)
        label_dst_path = os.path.join(label_dst, label_filename)

        if os.path.exists(label_src_path):
            shutil.copy2(label_src_path, label_dst_path)
            copied_count += 1
            print(f"Copied: {label_filename}")
        else:
            print(f"Warning: Label missing for {img_name}")

    print(f"Total labels copied: {copied_count}")
    return copied_count

In [ ]:
train_imgs="/content/drive/MyDrive/yolo_data/images/train"
val_imgs="/content/drive/MyDrive/yolo_data/images/val"
label_src="/content/drive/MyDrive/yolo_data/labelfull"

In [ ]:
copy_labels_for_images(train_imgs, label_src, "/content/drive/MyDrive/yolo_data/labels/train")
copy_labels_for_images(val_imgs, label_src, "/content/drive/MyDrive/yolo_data/labels/val")

# **Training & testing YOLOv5**

In [ ]:
model.train(data="/content/drive/MyDrive/yolo_data/mask_detection.yaml", epochs=50, imgsz=640, batch=8)

In [ ]:
model_custom = YOLO("/content/drive/MyDrive/yolo_data/best.pt") #possible data leakage

In [ ]:
model_custom2 = YOLO("/content/drive/MyDrive/yolo_data/yolo_training_results2/train2/weights/best.pt")

In [ ]:
destination_dir = '/content/drive/MyDrive/yolo_data/yolo_training_results2'

In [ ]:
os.makedirs(os.path.dirname(destination_dir), exist_ok=True)

In [ ]:
source_dir = '/content/runs'
shutil.copytree(source_dir, destination_dir)

In [ ]:
results = model_custom2.predict(
    source='/content/drive/MyDrive/yolo_data/images/val',
    save=True,
    save_txt=True,
    save_conf=True,
    project='prediction2',
    name='yolov5m_predictions',
    exist_ok=True
)

In [ ]:
!cp -r /content/prediction2 /content/drive/MyDrive/yolo_data/

# **Testing YOLO on custom dataset, TP, TN, FP, FN**

In [ ]:
val_image_folder = Path("/content/drive/MyDrive/yolo_data/images/val")

val_images = list(val_image_folder.glob("*.jpg")) + list(val_image_folder.glob("*.png"))

random.shuffle(val_images)
selected_images = val_images[:10]

for img_path in selected_images:
    original_img = cv2.imread(str(img_path))
    results = model_custom(img_path)
    img_with_boxes = results[0].plot()

    plt.figure(figsize=(10, 8))
    plt.imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title(f"Original image: {img_path.name}")
    plt.show()

    plt.figure(figsize=(10, 8))
    plt.imshow(cv2.cvtColor(img_with_boxes, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title(f"Predictions for: {img_path.name}")
    plt.show()

In [ ]:
def load_yolo_labels(txt_path, img_shape):
    h, w = img_shape[:2]
    boxes = []
    if not txt_path.exists():
        return boxes
    with open(txt_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            cls_id, cx, cy, bw, bh = map(float, parts)
            xmin = int((cx - bw / 2) * w)
            ymin = int((cy - bh / 2) * h)
            xmax = int((cx + bw / 2) * w)
            ymax = int((cy + bh / 2) * h)
            boxes.append([int(cls_id), xmin, ymin, xmax, ymax])
    return boxes

def calculate_iou(box1, box2):
    xA = max(box1[1], box2[1])
    yA = max(box1[2], box2[2])
    xB = min(box1[3], box2[3])
    yB = min(box1[4], box2[4])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    box1Area = (box1[3] - box1[1]) * (box1[4] - box1[2])
    box2Area = (box2[3] - box2[1]) * (box2[4] - box2[2])
    unionArea = float(box1Area + box2Area - interArea)
    return interArea / unionArea if unionArea != 0 else 0

In [ ]:
def visualize_predictions_with_stats(
    model, val_image_folder, labels_folder, classes,
    num_images=10, iou_thresh=0.5, conf_thresh=0.25
):
    val_images = list(Path(val_image_folder).glob("*.jpg")) + list(Path(val_image_folder).glob("*.png"))
    random.shuffle(val_images)
    selected_images = val_images[:num_images]

    for img_path in selected_images:
        original_img = cv2.imread(str(img_path))
        image_h, image_w = original_img.shape[:2]

        txt_path = Path(labels_folder) / (img_path.stem + ".txt")
        gt_boxes = load_yolo_labels(txt_path, original_img.shape)

        results = model(img_path)[0]
        pred_boxes = []

        for box in results.boxes:
            if box.conf.item() < conf_thresh:
                continue
            cls = int(box.cls.item())
            xmin, ymin, xmax, ymax = map(int, box.xyxy[0])
            pred_boxes.append([cls, xmin, ymin, xmax, ymax])

        matched_gt = set()
        matched_pred = set()
        fn_count = [0] * len(classes)
        fp_count = [0] * len(classes)

        for gi, gt in enumerate(gt_boxes):
            best_iou = 0
            best_pi = -1
            for pi, pred in enumerate(pred_boxes):
                if pred[0] != gt[0] or pi in matched_pred:
                    continue
                iou = calculate_iou(gt, pred)
                if iou > best_iou:
                    best_iou = iou
                    best_pi = pi
            if best_iou >= iou_thresh:
                matched_gt.add(gi)
                matched_pred.add(best_pi)
            else:
                fn_count[gt[0]] += 1

        for pi, pred in enumerate(pred_boxes):
            if pi not in matched_pred:
                fp_count[pred[0]] += 1

        img_gt = original_img.copy()
        for cls, xmin, ymin, xmax, ymax in gt_boxes:
            cv2.rectangle(img_gt, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            cv2.putText(img_gt, classes[cls], (xmin, ymin - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        img_pred = original_img.copy()
        for cls, xmin, ymin, xmax, ymax in pred_boxes:
            cv2.rectangle(img_pred, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)
            cv2.putText(img_pred, classes[cls], (xmin, ymin - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        plt.figure(figsize=(14, 6))
        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(img_gt, cv2.COLOR_BGR2RGB))
        plt.title(f"GT for {img_path.name}")
        plt.axis("off")

        plt.subplot(1, 2, 2)
        plt.imshow(cv2.cvtColor(img_pred, cv2.COLOR_BGR2RGB))
        plt.title(f"Predictions for {img_path.name}")
        plt.axis("off")
        plt.show()

        print(f"Stats for: {img_path.name}")
        for i, class_name in enumerate(classes):
            if fn_count[i] > 0 or fp_count[i] > 0:
                print(f" - {class_name}: FN={fn_count[i]}, FP={fp_count[i]}")
        print("—" * 40)

In [ ]:
visualize_predictions_with_stats(
    model=model_custom2,
    val_image_folder="/content/drive/MyDrive/yolo_data/images/val",
    labels_folder="/content/drive/MyDrive/yolo_data/labels/val",
    classes=classes
)

In [ ]:
results = model_custom("/content/drive/MyDrive/yolo_data/test_data/fmask4.jpg")
img_with_boxes = results[0].plot()

plt.figure(figsize=(10, 8))
plt.imshow(cv2.cvtColor(img_with_boxes, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Predicted Bounding Boxes")
plt.show()

In [ ]:
def compute_confusion_stats(model):
    model = model
    metrics = model.val()

    cm = metrics.confusion_matrix.matrix

    class_names = metrics.names
    stats = []

    for i, class_name in enumerate(class_names):
        TP = cm[i, i]
        FP = cm[:, i].sum() - TP
        FN = cm[i, :].sum() - TP
        TN = cm.sum() - (TP + FP + FN)
        stats.append({
            'Class': class_name,
            'TP': int(TP),
            'FP': int(FP),
            'FN': int(FN),
            'TN': int(TN),
        })

    df = pd.DataFrame(stats)
    return df, metrics

In [ ]:
def plot_confusion_stats(df_stats, class_names):
    plt.style.use('default')

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    colors = {'TP': '#22c55e', 'FP': '#ef4444', 'FN': '#f97316', 'TN': '#3b82f6'}

    for i, (idx, row) in enumerate(df_stats.iterrows()):
        if i < 3:
            ax = axes[i]
            values = [row['TP'], row['FP'], row['FN'], row['TN']]
            labels = ['TP', 'FP', 'FN', 'TN']
            colors_list = [colors[label] for label in labels]

            non_zero_data = [(val, lab, col) for val, lab, col in zip(values, labels, colors_list) if val > 0]
            if non_zero_data:
                values_nz, labels_nz, colors_nz = zip(*non_zero_data)

                wedges, texts, autotexts = ax.pie(values_nz, labels=labels_nz, colors=colors_nz,
                                                 autopct=lambda pct: f'{pct:.1f}%' if pct > 5 else '',
                                                 startangle=90)

                class_name = class_names[i] if i < len(class_names) else f'Class {i}'
                ax.set_title(f'{class_name}\nTotal: {sum(values)}', fontweight='bold')

                for autotext in autotexts:
                    autotext.set_color('white')
                    autotext.set_fontweight('bold')
                    autotext.set_fontsize(9)
            else:
                class_name = class_names[i] if i < len(class_names) else f'Class {i}'
                ax.text(0.5, 0.5, f'{class_name}\nNo Data',
                       ha='center', va='center', transform=ax.transAxes)
                ax.set_title(f'{class_name}', fontweight='bold')

    for i in range(len(df_stats), 3):
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
df_stats, metrics = compute_confusion_stats(model_custom)

print(df_stats)
print("\n")

plot_confusion_stats(df_stats, classes)

In [ ]:
df_stats, metrics = compute_confusion_stats(model_custom2)

print(df_stats)
print("\n")

plot_confusion_stats(df_stats, classes)

# **LIME integraion for explanation**

In [ ]:
!pip install lime
!pip install scikit-image

In [ ]:
from lime import lime_image
from skimage.segmentation import mark_boundaries
from PIL import Image

In [ ]:
class YOLOWrapper:
    def __init__(self, model, target_class_name):
        self.model = model
        self.target_class_name = target_class_name

    def predict(self, images):
        preds = []
        for img in images:

            img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            results = self.model(img_bgr)[0]

            class_names = results.names
            detections = results.boxes
            if detections is not None:
                classes = detections.cls.cpu().numpy()
                scores = detections.conf.cpu().numpy()
                labels = [class_names[int(c)] for c in classes]

                if self.target_class_name in labels:
                    preds.append([1])
                    continue
            preds.append([0])
        return np.array(preds)

In [ ]:
num_selected_img=1

In [ ]:
def get_images_with_class(model, target_class_name, image_folder, num_images=num_selected_img, image_size=640):

    image_folder = Path(image_folder)
    image_paths = list(image_folder.glob("*.jpg")) + list(image_folder.glob("*.png"))
    random.shuffle(image_paths)

    selected = []

    for img_path in image_paths:
        img = np.array(Image.open(img_path).convert("RGB").resize((image_size, image_size)))
        img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        results = model(img_bgr)[0]

        if results.boxes is not None:
            predicted_classes = [results.names[int(c)] for c in results.boxes.cls.cpu().numpy()]
            if target_class_name in predicted_classes:
                selected.append(img_path)

        if len(selected) >= num_images:
            break

    return selected

In [ ]:
def get_images_with_class_from_list(model, target_class_name, image_list, image_folder,  image_size=640):

    image_folder = Path(image_folder)
    image_paths = [image_folder / fname for fname in image_list]
    random.shuffle(image_paths)

    selected = []

    for img_path in image_paths:
        img = np.array(Image.open(img_path).convert("RGB").resize((image_size, image_size)))
        img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        results = model(img_bgr)[0]

        if results.boxes is not None:
            predicted_classes = [results.names[int(c)] for c in results.boxes.cls.cpu().numpy()]
            if target_class_name in predicted_classes:
                selected.append(img_path)

    return selected

In [ ]:
image_folder = "/content/drive/MyDrive/yolo_data/images/val"
image_list = [f"maksssksksss748.png"]
target_class = classes[2]
result = get_images_with_class_from_list(model_custom2 , target_class, image_list, image_folder)

In [ ]:
val_image_folder = Path("/content/drive/MyDrive/yolo_data/images/val")

In [ ]:
def detailed_lime_explanation(model, target_class_name, image_paths, image_size=640):
    explainer = lime_image.LimeImageExplainer()
    wrapped_model = YOLOWrapper(model, target_class_name)

    for img_path in image_paths:
        print(f"\nImage analysis: {img_path.name}")
        print("=" * 50)

        img = np.array(Image.open(img_path).convert("RGB").resize((image_size, image_size)))

        segmentation_configs = [
            {"n_segments": 50, "compactness": 15, "name": "Basic (50 segments)"},
            {"n_segments": 100, "compactness": 10, "name": "Detailed (100 segments)"},
            {"n_segments": 200, "compactness": 5, "name": "Very detailed (200 segments)"}
        ]

        fig, axes = plt.subplots(len(segmentation_configs), 5, figsize=(25, 5 * len(segmentation_configs)))
        if len(segmentation_configs) == 1:
            axes = axes.reshape(1, -1)

        for i, config in enumerate(segmentation_configs):
            explanation = explainer.explain_instance(
                image=img,
                classifier_fn=wrapped_model.predict,
                top_labels=1,
                hide_color=0,
                num_samples=1000,
                segmentation_fn=lambda x: slic(x, n_segments=config["n_segments"], compactness=config["compactness"])
            )

            importance_values = dict(explanation.local_exp[explanation.top_labels[0]])

            full_segments = explanation.segments
            heatmap = np.zeros(img.shape[:2])

            for segment_id, importance in importance_values.items():
                segment_mask = (full_segments == segment_id)
                heatmap[segment_mask] = importance

            axes[i, 0].imshow(img)
            axes[i, 0].set_title(f'Original\n{config["name"]}')
            axes[i, 0].axis('off')

            temp, mask = explanation.get_image_and_mask(
                label=explanation.top_labels[0],
                positive_only=True,
                hide_rest=False,
                num_features=5
            )
            axes[i, 1].imshow(mark_boundaries(temp, mask))
            axes[i, 1].set_title('Top 5 Positive')
            axes[i, 1].axis('off')

            temp, mask = explanation.get_image_and_mask(
                label=explanation.top_labels[0],
                positive_only=False,
                hide_rest=False,
                num_features=10
            )
            axes[i, 2].imshow(mark_boundaries(temp, mask))
            axes[i, 2].set_title('Top 10 Mixed')
            axes[i, 2].axis('off')

            im1 = axes[i, 3].imshow(heatmap, cmap='RdBu_r', alpha=0.8,
                                   vmin=np.min(list(importance_values.values())),
                                   vmax=np.max(list(importance_values.values())))
            axes[i, 3].imshow(img, alpha=0.4)
            axes[i, 3].set_title('Raw Importance\n(Red=+, Blue=-)')
            axes[i, 3].axis('off')
            plt.colorbar(im1, ax=axes[i, 3], fraction=0.046, pad=0.04)

            if np.max(heatmap) > np.min(heatmap):
                heatmap_norm = (heatmap - np.min(heatmap)) / (np.max(heatmap) - np.min(heatmap))
            else:
                heatmap_norm = heatmap

            im2 = axes[i, 4].imshow(heatmap_norm, cmap='viridis', alpha=0.8)
            axes[i, 4].imshow(img, alpha=0.4)
            axes[i, 4].set_title('Normalized\n(Purple=Low, Yellow=High)')
            axes[i, 4].axis('off')
            plt.colorbar(im2, ax=axes[i, 4], fraction=0.046, pad=0.04)

        plt.tight_layout()
        plt.show()

In [ ]:
def compare_predictions_and_explanations(model, target_class_name, image_paths):
    wrapped_model = YOLOWrapper(model, target_class_name)

    for img_path in image_paths:
        img = np.array(Image.open(img_path).convert("RGB").resize((640, 640)))

        prediction = wrapped_model.predict([img])[0]
        confidence = prediction[0]

        print(f"\nImage: {img_path.name}")
        print(f"Model confidence for class '{target_class_name}': {confidence:.4f}")

In [ ]:
target_class = classes[1]

In [ ]:
selected_images = get_images_with_class(
    model=model_custom2,
    target_class_name=target_class,
    image_folder=val_image_folder,
    num_images=num_selected_img
)

In [ ]:
print(selected_images)

In [ ]:
print("\n3. Detailed LIME analysis of correct predictions:")
print("-" * 50)
compare_predictions_and_explanations(model_custom2, target_class, selected_images)
detailed_lime_explanation(model_custom2, target_class, selected_images)

In [ ]:
print("\n3. Detailed LIME analysis of correct predictions:") #with fp
print("-" * 50)
compare_predictions_and_explanations(model_custom2, target_class, result)
detailed_lime_explanation(model_custom2, target_class, result)

# **Dataset correction with augmentation to try to better detect improperly worn masks**

In [ ]:
import albumentations as A
from scipy import ndimage

In [ ]:
class ContrastBasedMaskAugmentation:
    def __init__(self):
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    def get_face_region(self, image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, 1.1, 4)

        if len(faces) > 0:
            x, y, w, h = max(faces, key=lambda f: f[2] * f[3])
            return {'x': x, 'y': y, 'w': w, 'h': h}
        return None

    def analyze_face_contrast(self, image, face_region):
        if face_region is None:
            return None

        x, y, w, h = face_region['x'], face_region['y'], face_region['w'], face_region['h']

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        face_gray = gray[y:y+h, x:x+w]

        zones = {
            'forehead': face_gray[0:int(h*0.3), :],
            'eyes': face_gray[int(h*0.2):int(h*0.5), :],
            'nose': face_gray[int(h*0.35):int(h*0.65), :],
            'mouth': face_gray[int(h*0.6):int(h*0.8), :],
            'chin': face_gray[int(h*0.75):h, :],
            'left_cheek': face_gray[int(h*0.4):int(h*0.7), 0:int(w*0.3)],
            'right_cheek': face_gray[int(h*0.4):int(h*0.7), int(w*0.7):w],
        }

        contrast_stats = {}

        for zone_name, zone_img in zones.items():
            if zone_img.size == 0:
                continue

            mean_intensity = np.mean(zone_img)
            std_intensity = np.std(zone_img)

            local_contrast = np.max(zone_img) - np.min(zone_img)

            grad_x = cv2.Sobel(zone_img, cv2.CV_64F, 1, 0, ksize=3)
            grad_y = cv2.Sobel(zone_img, cv2.CV_64F, 0, 1, ksize=3)
            gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)
            avg_gradient = np.mean(gradient_magnitude)

            contrast_stats[zone_name] = {
                'mean': mean_intensity,
                'std': std_intensity,
                'local_contrast': local_contrast,
                'gradient': avg_gradient,
                'contrast_ratio': std_intensity / (mean_intensity + 1e-6)
            }

        return contrast_stats, face_gray

    def detect_incorrect_mask_by_contrast(self, contrast_stats):
        if not contrast_stats:
            return False, "no_face"

        nose_contrast = contrast_stats.get('nose', {}).get('contrast_ratio', 0)
        mouth_contrast = contrast_stats.get('mouth', {}).get('contrast_ratio', 0)
        chin_contrast = contrast_stats.get('chin', {}).get('contrast_ratio', 0)

        forehead_contrast = contrast_stats.get('forehead', {}).get('contrast_ratio', 0)
        left_cheek_contrast = contrast_stats.get('left_cheek', {}).get('contrast_ratio', 0)
        right_cheek_contrast = contrast_stats.get('right_cheek', {}).get('contrast_ratio', 0)

        masked_avg_contrast = np.mean([forehead_contrast, left_cheek_contrast, right_cheek_contrast])

        nose_suspicious = nose_contrast > masked_avg_contrast * 1.5
        mouth_suspicious = mouth_contrast > masked_avg_contrast * 1.3
        chin_suspicious = chin_contrast > masked_avg_contrast * 1.2

        is_incorrect = nose_suspicious or mouth_suspicious or chin_suspicious

        reason = []
        if nose_suspicious: reason.append("exposed_nose")
        if mouth_suspicious: reason.append("exposed_mouth")
        if chin_suspicious: reason.append("exposed_chin")

        return is_incorrect, reason

    def create_contrast_based_fake_masks(self, correct_mask_image):
        face_region = self.get_face_region(correct_mask_image)
        if face_region is None:
            return correct_mask_image

        img_copy = correct_mask_image.copy()
        x, y, w, h = face_region['x'], face_region['y'], face_region['w'], face_region['h']

        nose_y1 = y + int(h * 0.35)
        nose_y2 = y + int(h * 0.65)
        nose_x1 = x + int(w * 0.3)
        nose_x2 = x + int(w * 0.7)

        nose_region = img_copy[nose_y1:nose_y2, nose_x1:nose_x2]

        brightened = cv2.convertScaleAbs(nose_region, alpha=1.3, beta=20)

        brightened[:,:,0] = np.clip(brightened[:,:,0] * 0.9, 0, 255)
        brightened[:,:,1] = np.clip(brightened[:,:,1] * 1.1, 0, 255)
        brightened[:,:,2] = np.clip(brightened[:,:,2] * 1.2, 0, 255)

        noise = np.random.normal(0, 8, brightened.shape).astype(np.int16)
        brightened = np.clip(brightened.astype(np.int16) + noise, 0, 255).astype(np.uint8)

        img_copy[nose_y1:nose_y2, nose_x1:nose_x2] = brightened

        return img_copy

    def augment_for_contrast_detection(self, image):

        enhance_contrast = A.Compose([
            A.CLAHE(clip_limit=3.0, tile_grid_size=(8, 8), p=0.9),
            A.UnsharpMask(blur_limit=5, alpha=0.3, p=0.7),
        ])

        edge_enhance = A.Compose([
          A.Sharpen(alpha=(0.1, 0.3), lightness=(0.8, 1.2), p=0.8),
          A.Emboss(alpha=(0.05, 0.15), strength=(0.3, 0.7), p=0.3),
        ])


        color_contrast = A.Compose([
            A.HueSaturationValue(
                hue_shift_limit=8,
                sat_shift_limit=15,
                val_shift_limit=15,
                p=0.7
            ),
            A.RandomBrightnessContrast(
                brightness_limit=0.15,
                contrast_limit=0.25,
                p=0.8
            ),
        ])

        image = enhance_contrast(image=image)['image']
        image = edge_enhance(image=image)['image']
        image = color_contrast(image=image)['image']

        return image


In [ ]:
output_dir_aug="/content/drive/MyDrive/yolo_data/dataset_aug1"

In [ ]:
 os.makedirs(f"{output_dir_aug}", exist_ok=True)

In [ ]:
def process_yolo_dataset_with_contrast_universal(dataset_dir, output_dir, analyze_existing=True):
    augmenter = ContrastBasedMaskAugmentation()

    os.makedirs(f"{output_dir}/images/train", exist_ok=True)
    os.makedirs(f"{output_dir}/images/val", exist_ok=True)
    os.makedirs(f"{output_dir}/labels/train", exist_ok=True)
    os.makedirs(f"{output_dir}/labels/val", exist_ok=True)

    labels_folder = None
    possible_labels_names = ['labels', 'annotations', 'annotation', 'ann']

    for name in possible_labels_names:
        if os.path.exists(os.path.join(dataset_dir, name)):
            labels_folder = name
            print(f"Found labels folder: {name}")
            break

    if not labels_folder:
        print("Warning: No labels folder found! Looking for: labels, annotations, annotation, ann")
        return None

    stats = {
        'total_processed': 0,
        'incorrect_masks_found': 0,
        'synthetic_created': 0,
        'contrast_analysis': [],
        'train_stats': {'processed': 0, 'augmented': 0},
        'val_stats': {'processed': 0, 'augmented': 0}
    }

    for split in ['train', 'val']:
        images_dir = os.path.join(dataset_dir, 'images', split)
        labels_dir = os.path.join(dataset_dir, labels_folder, split)

        if not os.path.exists(images_dir):
            if split == 'train':
                images_dir = os.path.join(dataset_dir, 'images')
                labels_dir = os.path.join(dataset_dir, labels_folder)
                print(f"Using main directories (no train/val split found)")
            else:
                continue

        if not os.path.exists(images_dir) or not os.listdir(images_dir):
            print(f"Skipping {split} - directory empty or doesn't exist")
            continue

        print(f"\nProcessing {split} split...")
        print(f"Images dir: {images_dir}")
        print(f"Labels dir: {labels_dir}")

        image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        print(f"Found {len(image_files)} image files")

        for img_file in image_files:
            print(f"Analyzing image: {img_file}")
            img_path = os.path.join(images_dir, img_file)
            label_path = os.path.join(labels_dir, img_file.rsplit('.', 1)[0] + '.txt')

            image = cv2.imread(img_path)
            if image is None:
                print(f"  Warning: Could not load image {img_file}")
                continue

            face_region = augmenter.get_face_region(image)
            if face_region:
                contrast_stats, face_gray = augmenter.analyze_face_contrast(image, face_region)
                is_incorrect, reasons = augmenter.detect_incorrect_mask_by_contrast(contrast_stats)

                print(f"  Contrast analysis: {'incorrect' if is_incorrect else 'correct'}")
                if is_incorrect:
                    print(f"  Reasons: {', '.join(reasons)}")

                stats['contrast_analysis'].append({
                    'file': img_file,
                    'split': split,
                    'is_incorrect': is_incorrect,
                    'reasons': reasons,
                    'nose_contrast': contrast_stats.get('nose', {}).get('contrast_ratio', 0),
                    'mouth_contrast': contrast_stats.get('mouth', {}).get('contrast_ratio', 0)
                })
            else:
                print(f"  No face detected in {img_file}")

            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    labels = f.readlines()

                has_incorrect_mask = any('2 ' in label for label in labels)

                if has_incorrect_mask:
                    stats['incorrect_masks_found'] += 1
                    stats[f'{split}_stats']['augmented'] += 1

                    augmented_image = augmenter.augment_for_contrast_detection(image)

                    aug_img_path = f"{output_dir}/images/{split}/aug_{img_file}"
                    aug_label_path = f"{output_dir}/labels/{split}/aug_{img_file.rsplit('.', 1)[0]}.txt"

                    cv2.imwrite(aug_img_path, augmented_image)
                    with open(aug_label_path, 'w') as f:
                        f.writelines(labels)

                    print(f"Created augmented data: aug_{img_file}")
                    stats['synthetic_created'] += 1
                else:
                    print(f"  No incorrect masks found, skipping augmentation")
            else:
                print(f"  Warning: Label file not found: {label_path}")

            stats['total_processed'] += 1
            stats[f'{split}_stats']['processed'] += 1

        if split == 'train' and images_dir == os.path.join(dataset_dir, 'images'):
            break

    print(f"Total images processed: {stats['total_processed']}")
    print(f"Images with incorrect masks: {stats['incorrect_masks_found']}")
    print(f"Synthetic images created: {stats['synthetic_created']}")
    print(f"Train - Processed: {stats['train_stats']['processed']}, Augmented: {stats['train_stats']['augmented']}")

    return stats

In [ ]:
def copy_folder_contents(src_dir, dst_dir):
    if not os.path.exists(src_dir):
        print(f"source do not exist: {src_dir}")
        return

    os.makedirs(dst_dir, exist_ok=True)

    for item in os.listdir(src_dir):
        s = os.path.join(src_dir, item)
        d = os.path.join(dst_dir, item)

        if os.path.isdir(s):
            shutil.copytree(s, d, dirs_exist_ok=True)
        else:
            shutil.copy2(s, d)

    print(f"content was copied from: \n{src_dir}\n to: \n{dst_dir}")

In [ ]:
dataset_dir = "/content/drive/MyDrive/yolo_data"
output_dir_aug = "/content/drive/MyDrive/yolo_data/dataset_aug1"

stats = process_yolo_dataset_with_contrast_universal(dataset_dir, output_dir_aug)

In [ ]:
copy_folder_contents("/content/drive/MyDrive/yolo_data/images/val","/content/drive/MyDrive/yolo_data/dataset_aug1/images/val" )

In [ ]:
copy_folder_contents("/content/drive/MyDrive/yolo_data/labels/val","/content/drive/MyDrive/yolo_data/dataset_aug1/images/val" )

# **Training YOLO with dataset with augmentation**

In [ ]:
model2 = YOLO('yolov5m.pt')

In [ ]:
destination_dir = '/content/drive/MyDrive/yolo_data/yolo_training_results3'
os.makedirs(os.path.dirname(destination_dir), exist_ok=True)

In [ ]:
model2.train(data="/content/drive/MyDrive/yolo_data/mask_detection_aug.yaml", epochs=50, imgsz=640)

In [ ]:
model_custom3 = YOLO("/content/drive/MyDrive/yolo_data/yolo_training_results3/train3/weights/best.pt")

In [ ]:
results = model_custom3.predict(
    source='/content/drive/MyDrive/yolo_data/images/val',
    save=True,
    save_txt=True,
    save_conf=True,
    project='prediction3',
    name='yolov5m_predictions',
    exist_ok=True
)

In [ ]:
!cp -r /content/prediction3 /content/drive/MyDrive/yolo_data/

# **augmentation YOLO testing, TP, TN, FP, FN**

In [ ]:
df_stats, metrics = compute_confusion_stats(model_custom3)

print(df_stats)
print("\n")

plot_confusion_stats(df_stats, classes)

# **Improving parameters in training**

In [ ]:
model3 = YOLO('yolov5m.pt')

In [ ]:
model3.train(
    data="/content/drive/MyDrive/yolo_data/mask_detection.yaml",
    epochs=100,

    imgsz=640,
    batch=16,
    box=5.0,
    cls=1.0
)

In [ ]:
destination_dir = '/content/drive/MyDrive/yolo_data/yolo_training_results4'
os.makedirs(os.path.dirname(destination_dir), exist_ok=True)

In [ ]:
source_dir = '/content/runs'
shutil.copytree(source_dir, destination_dir, dirs_exist_ok=True)

In [ ]:
model_custom4 = YOLO("/content/drive/MyDrive/yolo_data/yolo_training_results4/train/weights/best.pt")

In [ ]:
df_stats, metrics = compute_confusion_stats(model_custom4)

print(df_stats)
print("\n")

plot_confusion_stats(df_stats, classes)